In [2]:
# Anonymisera idrottarnas namn
import pandas as pd
import hashlib as hl

df = pd.read_csv("../Data/athlete_events.csv")
df["Name"] = df["Name"].astype(str)
df["Name"] = df["Name"].apply(lambda athlete_name: hl.sha256(athlete_name.encode()).hexdigest())
#df.head()

In [3]:
hungary = pd.read_csv("../Data/athlete_events.csv")
hungary = hungary[ hungary["NOC"]=="HUN" ]


In [4]:
# Remove all individual medals in group sports

def  unique_medals( df ):
    set_year_medal_event = set()
    for idx, row in df.dropna( subset=["Medal"] ).iterrows():
        set_year_medal_event.add(f" {row['Year']}|{row['Event']}|{row['Medal']} ")
    len(set_year_medal_event)

    year = [ row.split("|")[0][1:] for row in set_year_medal_event ]
    event = [ row.split("|")[1] for row in set_year_medal_event ]
    medal = [ row.split("|")[2][:-1] for row in set_year_medal_event ]

    df_um = pd.DataFrame(   
        {"Year":year, "Medal":medal, "Event":event}
    ) 

    return df_um
hungary_medals_per_event = unique_medals( hungary )

In [5]:
import plotly_express as px

def medals_only( df ):
    df.groupby("Year")["Medal"].count()
    df["Gold"] = df["Medal"] == "Gold" 
    df["Silver"] = df["Medal"] == "Silver" 
    df["Bronze"] = df["Medal"] == "Bronze" 

    return df[["Year","Gold","Silver","Bronze"]].groupby("Year").sum()

hungary_medals = medals_only( hungary_medals_per_event  )

fig = px.line(hungary_medals, y=["Gold", "Silver", "Bronze"], color_discrete_sequence=['Gold', 'Silver', 'Brown'])


fig.show()
hungary_medals


,Gold,Silver,Bronze
Year,,,
1896,2,1,3
1900,1,2,2
1904,2,1,1
1906,2,5,3
1908,3,4,2
1912,3,2,3
1924,2,4,4
1928,5,5,0
1932,6,5,6


In [6]:
# Uppgift 2 

# All countries except HUN
df_um = unique_medals( df[ df.NOC != "HUN"] )
df_mo = medals_only( df_um )
#df_mo.drop(hungary_medals.index, inplace=True)
df_mo = df_mo[ df_mo.index.isin( hungary_medals.index   ) ]




In [7]:

len(df_mo)

hungary_medals["Gold (%)"] = hungary_medals.apply(lambda x: round(x["Gold"]/df_mo.loc[x.name]["Gold"]*100, 1), axis=1) 
hungary_medals["Silver (%)"] = hungary_medals.apply(lambda x: round(x["Silver"]/df_mo.loc[x.name]["Silver"]*100, 1), axis=1) 
hungary_medals["Bronze (%)"] = hungary_medals.apply(lambda x: round(x["Gold"]/df_mo.loc[x.name]["Gold"]*100, 1), axis=1) 
hungary_medals



#fig = px.pie( values=[hungary_medals["Gold (%)"]., hungary_medals.loc[0]["Gold"] ])
#fig.show()
import plotly.graph_objects as go
data=[ 

        go.Bar( x=hungary_medals.index, y=hungary_medals["Bronze (%)"], name="Bronze",  #orientation="h", 
            marker=dict( color='rgba(232, 140, 70, 0.6)', line=dict(color='rgba(232, 140, 70, 1.0)', width=2)
        )),

        go.Bar( x=hungary_medals.index, y=hungary_medals["Silver (%)"], name="Silver",  #orientation="h", 
            marker=dict( color='rgba(163, 173, 191, 0.6)', line=dict(color='rgba(163, 173, 191, 1.0)', width=2)
        )),


        go.Bar( x=hungary_medals.index, y=hungary_medals["Gold (%)"], name="Gold", #orientation="h",
            marker=dict( color='rgba(256, 186, 3, 0.6)', line=dict(color='rgba(256, 186, 3, 1.0)', width=2)
        )),
    ]
fig = go.Figure(data, layout=dict(barcornerradius=15))
fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
)
fig.show()


In [8]:
hungary_mpe = hungary_medals_per_event.groupby("Event", as_index=False)["Medal"].count()
hungary_mpe.set_index("Event", inplace=True)

hungary_mpe.sort_values("Medal", ascending=False, inplace=True)

hungary_mpe.head(30)

,Medal
Event,
"Fencing Men's Sabre, Individual",29
"Fencing Men's Sabre, Team",16
Water Polo Men's Water Polo,15
Modern Pentathlon Men's Individual,13
"Canoeing Men's Canadian Singles, 1,000 metres",10
"Canoeing Women's Kayak Doubles, 500 metres",10
"Fencing Women's Foil, Individual",9
"Wrestling Men's Featherweight, Greco-Roman",9
Athletics Men's Hammer Throw,9
